In [ ]:
import batoid
import os
import yaml
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
HSC_fn = os.path.join(batoid.datadir, "hsc", "HSC3.yaml")
config = yaml.load(open(HSC_fn))
telescope = batoid.parse.parse_optic(config['opticalSystem'])

In [ ]:
def pupil(xcos, ycos, nside=512):
    zcos = -np.sqrt(1.0-xcos**2-ycos**2)
    rays = batoid.rayGrid(20, 10, xcos, ycos, zcos, nside=nside, wavelength=600e-9, n=1.0)
    tf = telescope.traceFull(rays)
    w = np.logical_not(tf[-1]['out'].isVignetted)
    return tf[0]['in'].x[w]*np.sqrt(1-xcos**2), tf[0]['in'].y[w]*np.sqrt(1-ycos**2)

In [ ]:
fig = plt.figure(figsize=(12, 12))
ax = fig.add_subplot(111)
ax.scatter(*pupil(0.001,0), s=0.1)
fig.show()

In [ ]:
def spanRange(x, nside=512):
    xmin, xmax = np.min(x), np.max(x)
    xspan = xmax - xmin
    xmin = xmin - 0.8*xspan
    xmax = xmax + 0.8*xspan
    return np.linspace(xmin, xmax, nside)

In [ ]:
def pinhole(xcos, ycos, nside=256):
    # reset skips
    for item in telescope.itemDict.keys():
        telescope.itemDict[item].skip = False

    # First, need to determine where on the filter to constrain rays.  We'll use the average position of the 
    # pupil beam that would have intersected the filter.
    zcos = -np.sqrt(1.0-xcos**2-ycos**2)
    rays = batoid.rayGrid(20, 10, xcos, ycos, zcos, nside=nside, wavelength=600e-9, n=1.0)
    tf = telescope.traceFull(rays)
    surface = tf[[s['name'] for s in tf].index('F_entrance')]
    rs = batoid.trimVignetted(surface['out'])
    xmean, ymean = np.mean(rs.x), np.mean(rs.y)
    # Now we need to generate a bunch of rays that all pass through the above part of the filter, but over 
    # a range of angles.
    # What is the range of angles for the pupil beam?  
    vx = spanRange(rs.vx, nside=nside)
    vy = spanRange(rs.vy, nside=nside)
    vx, vy = np.meshgrid(vx, vy)
    vz = np.sqrt(1-vx*vx+vy*vy)
    # Now need to make a RayVector with appropriate x,y,vx,vy,...
    rv = batoid.RayVector([
        batoid.Ray(xmean, ymean, 0, vx_, vy_, vz_, 0, 600e-9)
        for vx_, vy_, vz_ in zip(vx.ravel(), vy.ravel(), vz.ravel())])
    # trace forward from filter.  So temporarily skip everything before the filter.
    before_items = ['SubaruHSC.TopRingStrutsAndCamera', 
                    'SubaruHSC.BottomRingStrutsFirstPass',
                    'SubaruHSC.PM',
                    'SubaruHSC.BottomRingStrutsSecondPass',
                    'SubaruHSC.HSC.G1',
                    'SubaruHSC.HSC.G2',
                    'SubaruHSC.HSC.ADC',
                    'SubaruHSC.HSC.G3',
                    'SubaruHSC.HSC.G4',
                    'SubaruHSC.HSC.G5',
                   ]
    for item in before_items:
        telescope.itemDict[item].skip = True
    forward_rays, _ = telescope.trace(rv, inCoordSys=telescope.itemDict['SubaruHSC.HSC.F'].coordSys)
    # reset skips
    for item in telescope.itemDict.keys():
        telescope.itemDict[item].skip = False
    # Now skip everything that happens *after* and including the filter
    after_items = ['SubaruHSC.HSC.F',
                   'SubaruHSC.HSC.W',
                   'SubaruHSC.HSC.D',
                  ]
    for item in after_items:
        telescope.itemDict[item].skip = True
    rv = batoid.RayVector([
        batoid.Ray(r.p0, -r.v, r.t0, r.wavelength)
        for r in rv]
    )
    reverse_rays, _ = telescope.traceReverse(rv, inCoordSys=telescope.itemDict['SubaruHSC.HSC.F'].coordSys)

    # reset skips
    for item in telescope.itemDict.keys():
        telescope.itemDict[item].skip = False
        
    w = np.where(np.logical_not(reverse_rays.isVignetted))[0]
    return forward_rays.x[w], forward_rays.y[w]

In [ ]:
fig = plt.figure(figsize=(12, 12))
ax = fig.add_subplot(111)
ax.scatter(*pinhole(0,0), s=1)
fig.show()

In [ ]:
def plot(xcos, ycos):
    fig = plt.figure(figsize=(6, 6))
    ax = fig.add_subplot(111)

    pux, puy = pupil(xcos, ycos)
    xspan = np.max(pux) - np.min(pux)
    yspan = np.max(puy) - np.min(puy)
    span = max(xspan, yspan)
    pux = (pux - np.mean(pux))/span
    puy = (puy - np.mean(puy))/span
    
    phx, phy = pinhole(xcos, ycos)
    xspan = np.max(phx) - np.min(phx)
    yspan = np.max(phy) - np.min(phy)
    span = max(xspan, yspan)

    phx = -(phx - np.mean(phx))/span
    phy = -(phy - np.mean(phy))/span

    ax.scatter(pux, puy, s=2, alpha=0.1, c='r', label='pupil')
    ax.scatter(phx, phy, s=2, alpha=0.2, c='b', label='pinhole')
    ax.legend()
    fig.show()

In [ ]:
plot(0, 0.74 * np.pi/180)

In [ ]:
def both(xcos, ycos):
    pux, puy = pupil(xcos, ycos)
    xspan = np.max(pux) - np.min(pux)
    yspan = np.max(puy) - np.min(puy)
    span = max(xspan, yspan)
    pux = (pux - np.mean(pux))/span
    puy = (puy - np.mean(puy))/span

    phx, phy = pinhole(xcos, ycos)
    xspan = np.max(phx) - np.min(phx)
    yspan = np.max(phy) - np.min(phy)
    span = max(xspan, yspan)
    phx = -(phx - np.mean(phx))/span
    phy = -(phy - np.mean(phy))/span

    return pux, puy, phx, phy

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(8,8))
for ax, ycos in zip(axes.ravel(), [0.0, 0.25, 0.5, 0.74]):
    pux, puy, phx, phy = both(0.0, ycos*np.pi/180)

    ax.scatter(pux, puy, s=2, alpha=0.1, c='r', label='pupil')
    ax.scatter(phx, phy, s=2, alpha=0.2, c='b', label='pinhole')
    ax.set_title(r"$\theta_y$ = {:5.2f}".format(ycos))
    ax.legend(loc="upper right")
fig.show()